# Day 7 - No Space Left On Device
## Data

In [1]:
example_data = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

print(example_data)

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k


In [2]:
import aocd
raw_data = aocd.get_data(year=2022, day=7)
print(raw_data)

$ cd /
$ ls
dir dscbfp
283653 fsdfddfv
dir mjzqq
241330 rcm.psp
dir sjbpgc
dir zfsbvs
$ cd dscbfp
$ ls
dir fgtvzpl
dir hgfrgbv
dir hmwqgjnl
dir jvr
dir lcvdgm
dir mmhtpz
dir wqc
dir znl
dir zph
dir zwlpm
$ cd fgtvzpl
$ ls
dir fvrghzfg
28513 lbbg.rhq
$ cd fvrghzfg
$ ls
212295 cjb.nwg
dir ftqs
$ cd ftqs
$ ls
250415 mmhtpz
$ cd ..
$ cd ..
$ cd ..
$ cd hgfrgbv
$ ls
86365 cdrgnzrz.hwf
175318 wqtmwb
$ cd ..
$ cd hmwqgjnl
$ ls
dir dscbfp
dir dwtfrgj
130223 fnl.whg
66339 mtcv
dir rgvvz
$ cd dscbfp
$ ls
146043 wpzr
$ cd ..
$ cd dwtfrgj
$ ls
dir jwmw
dir pntqg
$ cd jwmw
$ ls
dir dscbfp
243410 lbbg.phv
dir mmhtpz
$ cd dscbfp
$ ls
dir mmhtpz
$ cd mmhtpz
$ ls
dir mdrddz
$ cd mdrddz
$ ls
167704 fgjsq.bpb
$ cd ..
$ cd ..
$ cd ..
$ cd mmhtpz
$ ls
233626 dtcmsq.pdl
163642 jczs.rgg
111667 msfmjd.vlr
23137 ndhvh.jbq
$ cd ..
$ cd ..
$ cd pntqg
$ ls
68578 lnjvpcgq.zqs
62492 rcm.psp
$ cd ..
$ cd ..
$ cd rgvvz
$ ls
dir hmwqgjnl
dir jrjgnch
110656 lnjvpcgq.zqs
206537 mmhtpz.wgd
198736 msfmjd.vlr
110172 rrl.wq

## Parsing

In [70]:
def print_indent(indent):
    for i in range(indent):
            print('  ', end='')

            
class Node(object):
    def __repr__(self):
        return f"- {self.name} (size = {self.size})"
    
    def walk(self):
        yield self
        for child in self.children.values():
            yield from child.walk()
        

class Directory(Node):
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = {}
    
    @property
    def size(self):
        return sum(child.size for child in self.children.values())
    
    def print(self, indent = 0):
        print_indent(indent)    
        print(self)
        
        for child in self.children.values():
            child.print(indent + 1)
    
    def walk_dirs(self):
        yield self
        for child in self.children.values():
            try:
                yield from child.walk_dirs()
            except AttributeError:
                continue
    
    
class File(Node):
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.children = {} # Should be immutable
    
    def print(self, indent):
        print_indent(indent)
        print(self)
        

def parse_cd(cwd, args):
    """Parse a 'cd' command, and return the directory to move to."""
    dir_name = args[2] # directory name or .. for the parent.
    if dir_name == '..':
        return cwd.parent
    else:
        return cwd.children[dir_name]

    
def parse(input):
    """Parse the input commands and return the root directory object."""
    root = Directory('/')
    cwd = root
    
    # We can skip the first line because it is just "cd /"
    lines = input.split('\n')[1:]

    for line in lines:
        args = line.split(' ')
            
        if line[0] == '$':
            command = args[1]
            
            if command == 'cd':
                cwd = parse_cd(cwd, args)

        else:
            name = args[1]

            if args[0] == 'dir':
                new_node = Directory(name, cwd)
            else:
                new_node = File(name, int(args[0]))

            cwd.children[name] = new_node

    return root
            
            
example_tree = parse(example_data)
example_tree.print()
real_tree = parse(raw_data)
real_tree.print()

- / (size = 48381165)
  - a (size = 94853)
    - e (size = 584)
      - i (size = 584)
    - f (size = 29116)
    - g (size = 2557)
    - h.lst (size = 62596)
  - b.txt (size = 14848514)
  - c.dat (size = 8504156)
  - d (size = 24933642)
    - j (size = 4060174)
    - d.log (size = 8033020)
    - d.ext (size = 5626152)
    - k (size = 7214296)
- / (size = 44804833)
  - dscbfp (size = 43189731)
    - fgtvzpl (size = 491223)
      - fvrghzfg (size = 462710)
        - cjb.nwg (size = 212295)
        - ftqs (size = 250415)
          - mmhtpz (size = 250415)
      - lbbg.rhq (size = 28513)
    - hgfrgbv (size = 261683)
      - cdrgnzrz.hwf (size = 86365)
      - wqtmwb (size = 175318)
    - hmwqgjnl (size = 11152310)
      - dscbfp (size = 146043)
        - wpzr (size = 146043)
      - dwtfrgj (size = 1074256)
        - jwmw (size = 943186)
          - dscbfp (size = 167704)
            - mmhtpz (size = 167704)
              - mdrddz (size = 167704)
                - fgjsq.bpb (size = 16770

                - gtcg (size = 73017)
                  - pgzcm.qbz (size = 73017)
              - dwdnwdz (size = 235064)
                - hvgwfj (size = 235064)
                  - rnjjh.qnp (size = 235064)
              - rcm.psp (size = 291794)
          - djngvdp (size = 427283)
            - jczs.rgg (size = 264940)
            - rgvvz (size = 162343)
          - lnjvpcgq.zqs (size = 266568)
          - zdwhqb (size = 464641)
            - ggd (size = 233875)
            - ggqrt.pqn (size = 230766)
        - msfmjd.vlr (size = 23864)
        - nrbmbpm (size = 286035)
        - pwvgqth (size = 694584)
          - nrm (size = 135745)
            - hmwqgjnl.fwb (size = 135745)
          - rgvvz (size = 269675)
            - rcm.psp (size = 269675)
          - sgdsg.fbs (size = 289164)
        - rgvvz (size = 91650)
        - wllwhm (size = 14033)
          - rptrszg.lfh (size = 14033)
      - lbbg (size = 3267690)
        - fsz (size = 1222002)
          - cdsr (size = 29744)
     

## Part 1

In [71]:
def find_total_size_of_small_directories(root, max_size):
    """Returns the total size of all directories in the root directory with a maximum size."""    
    return sum(dir.size for dir in root.walk_dirs() if dir.size <= max_size)


assert find_total_size_of_small_directories(example_tree, 100000) == 95437
find_total_size_of_small_directories(real_tree, 100000)

1297683

## Part 2

In [83]:
def find_directory_to_delete(root, total_space, space_needed):
    """Returns the size of the smallest directory that, if deleted, would give enough space left."""
    minimum = root.size
    
    space_left = total_space - root.size
    
    for dir in root.walk_dirs():
        if space_left + dir.size >= space_needed and dir.size < minimum:
            minimum = dir.size
    return minimum

assert find_directory_to_delete(example_tree, 70000000, 30000000) == 24933642
find_directory_to_delete(real_tree, 70000000, 30000000)

5756764